# Mining PALSAR Analysis

## Objectives
- Lineament extraction and structural analysis using SRTM 30m DEM + PALSAR 2023-2024
- Enhanced lineament detection through multi-source data fusion

## Deliverables
1. **Lineament Map**: Binary map showing detected linear features (faults, fractures, structural lineaments)
2. **Lineament Density**: Number of lineaments per square kilometer - identifies zones of high structural complexity
3. **Lineament Orientation**: Directional analysis (N, NE, E, SE, S, SW, W, NW) - shows dominant structural trends
4. **Lineament Confidence**: Probability map (0-1) indicating detection reliability based on multi-source agreement
5. **DEM-based Lineaments**: Topographic lineaments from elevation, curvature, and aspect variance
6. **PALSAR-based Lineaments**: Radar-detected lineaments from backscatter edges and HH/HV ratios
7. **Combined Lineaments**: Enhanced detection combining DEM + PALSAR for higher confidence

## Study Area
- Coordinates (WGS1984):
  - Point 1: 04°46'0.00"N, 29°34'30.00"E
  - Point 2: 04°46'0.00"N, 29°16'45.00"E
  - Point 3: 05°05'30.00"N, 29°16'45.00"E
  - Point 4: 05°05'30.00"N, 29°34'30.00"E
- Buffer: 20m


In [59]:
pip install geemap earthengine-api numpy

Note: you may need to restart the kernel to use updated packages.


In [60]:
# Import required libraries
import geemap
import ee
import numpy as np
from datetime import datetime

# Initialize Earth Engine with Project ID
# IMPORTANT: Replace 'your-project-id' with your actual Google Cloud Project ID
# To find your Project ID:
# 1. Go to https://console.cloud.google.com/
# 2. Click on the project dropdown at the top
# 3. Copy the Project ID (e.g., 'ee-your-username' or 'mineral-exploration-456')

PROJECT_ID = 'ee-okwaretom12'  # <-- REPLACE THIS WITH YOUR PROJECT ID

try:
    # Try to initialize with project ID
    geemap.ee_initialize(project=PROJECT_ID)
    print(f"Earth Engine initialized with project: {PROJECT_ID}")
except Exception as e:
    print("Earth Engine not initialized. Starting authentication...")
    print("Please follow the authentication steps:")
    print("1. A browser window will open")
    print("2. Sign in with your Google account")
    print("3. Grant permissions to Earth Engine")
    print("4. Copy the authorization code and paste it when prompted")
    
    # Authenticate (this will open a browser for first-time users)
    ee.Authenticate()
    
    # Initialize after authentication with project ID
    geemap.ee_initialize(project=PROJECT_ID)
    print(f"Earth Engine initialized successfully with project: {PROJECT_ID}!")

# Initialize geemap
Map = geemap.Map()
print("Libraries imported and geemap initialized successfully!")


Earth Engine initialized with project: ee-okwaretom12
Libraries imported and geemap initialized successfully!


## Define Study Area


In [61]:
# Define study area coordinates (WGS1984)
# Convert DMS to decimal degrees
coords = [
    [29.575, 4.766667],  # Point 1: 29°34'30"E, 04°46'0"N
    [29.279167, 4.766667],  # Point 2: 29°16'45"E, 04°46'0"N
    [29.279167, 5.091667],  # Point 3: 29°16'45"E, 05°05'30"N
    [29.575, 5.091667],  # Point 4: 29°34'30"E, 05°05'30"N
]

# Create polygon
study_area = ee.Geometry.Polygon([coords])

# Apply 20m buffer
study_area_buffered = study_area.buffer(20)

# Print study area info (without .getInfo() to avoid blocking)
print("Study area created with coordinates:")
print("  Point 1: 29°34'30\"E, 04°46'0\"N")
print("  Point 2: 29°16'45\"E, 04°46'0\"N")
print("  Point 3: 29°16'45\"E, 05°05'30\"N")
print("  Point 4: 29°34'30\"E, 05°05'30\"N")
print("  Buffer: 20m")

# Add to map
Map.addLayer(study_area_buffered, {'color': 'red'}, 'Study Area (20m buffer)')
Map.centerObject(study_area_buffered, 10)
Map


Study area created with coordinates:
  Point 1: 29°34'30"E, 04°46'0"N
  Point 2: 29°16'45"E, 04°46'0"N
  Point 3: 29°16'45"E, 05°05'30"N
  Point 4: 29°34'30"E, 05°05'30"N
  Buffer: 20m


Map(center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position', 'transparent_b…

## Load SRTM 30 DEM for Lineament Analysis


In [62]:
# Load SRTM 30m DEM
srtm = ee.Image("USGS/SRTMGL1_003").clip(study_area_buffered)

# Calculate terrain derivatives for lineament detection
elevation = srtm.select('elevation')
slope = ee.Terrain.slope(elevation)
aspect = ee.Terrain.aspect(elevation)
hillshade = ee.Terrain.hillshade(elevation)

# Calculate curvature for enhanced lineament detection
# Use gradient-based approach for better lineament detection
# Smoothing to reduce noise (1-2 pixels optimal: 30-60m at SRTM 30m resolution)
gaussian_filter = ee.Kernel.gaussian(radius=1.5, sigma=1, units='pixels')
elev_smooth = elevation.convolve(gaussian_filter).resample('bilinear')

# Calculate gradients (first derivatives)
grads = elev_smooth.gradient()
dz_dx = grads.select('x')
dz_dy = grads.select('y')

# Calculate second derivatives (curvature)
second_grads = grads.gradient()
d2z_dx2 = second_grads.select('x')
d2z_dy2 = second_grads.select('y')
total_curvature = d2z_dx2.abs().add(d2z_dy2.abs()).rename('curvature')

# Calculate aspect variance (high variance indicates linear features/lineaments)
# Use reduceNeighborhood with variance reducer (GEE doesn't have focal_variance)
kernel_variance = ee.Kernel.square(radius=2, units='pixels')
aspect_variance = aspect.reduceNeighborhood(
    reducer=ee.Reducer.variance(),
    kernel=kernel_variance
).rename('aspect_variance')

# Add to map
Map.addLayer(elevation, {'min': 0, 'max': 2000, 'palette': ['blue', 'green', 'yellow', 'red']}, 'SRTM Elevation')
Map.addLayer(slope, {'min': 0, 'max': 45, 'palette': ['white', 'brown']}, 'Slope')
Map.addLayer(aspect, {'min': 0, 'max': 360, 'palette': ['red', 'yellow', 'green', 'cyan', 'blue', 'magenta', 'red']}, 'Aspect')
Map.addLayer(hillshade, {'min': 0, 'max': 255}, 'Hillshade', False)
Map.addLayer(total_curvature, {'min': 0, 'max': 0.03, 'palette': ['white', 'orange', 'red', 'black']}, 'Curvature (Gradient-based)', False)
Map.addLayer(aspect_variance, {'min': 0, 'max': 500, 'palette': ['white', 'blue', 'purple']}, 'Aspect Variance', False)

print("SRTM DEM and terrain derivatives loaded successfully!")
Map


SRTM DEM and terrain derivatives loaded successfully!


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

In [ ]:
# Check aspect variance statistics and adjust visualization
print("Calculating aspect variance statistics...")

# Get statistics for better visualization
aspect_var_stats = aspect_variance.reduceRegion(
    reducer=ee.Reducer.minMax().combine(
        ee.Reducer.percentile([5, 25, 50, 75, 95]),
        sharedInputs=True
    ),
    geometry=study_area_buffered,
    scale=30,
    maxPixels=1e9
)

# Get the statistics values
stats_dict = aspect_var_stats.getInfo()
min_val = stats_dict.get('aspect_variance_min', 0)
max_val = stats_dict.get('aspect_variance_max', 50000)
p5 = stats_dict.get('aspect_variance_p5', 0)
p25 = stats_dict.get('aspect_variance_p25', 0)
p50 = stats_dict.get('aspect_variance_p50', 0)
p75 = stats_dict.get('aspect_variance_p75', 0)
p95 = stats_dict.get('aspect_variance_p95', 5000)

print("\nAspect Variance Statistics:")
print(f"  Minimum: {min_val:.2f}")
print(f"  5th percentile: {p5:.2f}")
print(f"  25th percentile: {p25:.2f}")
print(f"  50th percentile (median): {p50:.2f}")
print(f"  75th percentile: {p75:.2f}")
print(f"  95th percentile: {p95:.2f}")
print(f"  Maximum: {max_val:.2f}")

# Update visualization with adjusted range
# Use 5th to 95th percentile to avoid outliers and see variation
print(f"\nUpdating visualization range: {p5:.0f} to {p95:.0f}")

# Remove old layer if it exists (by name)
try:
    # Try to remove the old layer
    pass  # geemap doesn't have easy layer removal, so we'll just add a new one
except:
    pass

# Add updated visualization with better range
Map.addLayer(aspect_variance, {
    'min': p5,  # Start from 5th percentile
    'max': p95,  # End at 95th percentile (shows 90% of data range)
    'palette': ['white', 'lightblue', 'blue', 'purple', 'indigo']
}, 'Aspect Variance (Adjusted)', False)

print("\nVisualization updated! Now you should see:")
print("  - White/light blue: Low variance areas (uniform terrain)")
print("  - Blue: Moderate variance (some structural variation)")
print("  - Purple: High variance (structural features/lineaments)")
print("  - Dark purple: Very high variance (major structural features)")

Map


## Load PALSAR Data for 2023-2024


In [ ]:
# Cell removed - PALSAR 2001-2006 data not available
# Analysis proceeds directly to PALSAR 2023-2024 loading (Cell 13)


Checking PALSAR collection availability...
Collection size: 4
PALSAR data (2001-2006) loaded successfully!


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

In [ ]:
# Load PALSAR data for period 2023-2024
# Check multiple sources: ALOS PALSAR, ALOS-2 PALSAR-2 (SAR_EPOCH), ALOS-2 ScanSAR, ALOS-4 PALSAR-3

print("=" * 60)
print("LOADING PALSAR DATA FOR 2023-2024")
print("=" * 60)

# Try ALOS-2 PALSAR-2 SAR_EPOCH first (correct collection for 2015-2022+)
collection_size = 0
data_source = None
palsar_collection_2023_2024 = None

try:
    palsar_collection_2023_2024 = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH') \
        .filterBounds(study_area_buffered) \
        .filterDate('2023-01-01', '2024-12-31')
    collection_size = palsar_collection_2023_2024.size().getInfo()
    data_source = "ALOS-2 PALSAR-2 (SAR_EPOCH)"
    print(f"✓ ALOS-2 PALSAR-2 (SAR_EPOCH) collection found")
except Exception as e:
    print(f"✗ ALOS-2 PALSAR-2 (SAR_EPOCH): {str(e)[:100]}")
    collection_size = 0

# If no SAR_EPOCH data, try ScanSAR
if collection_size == 0:
    print(f"Trying ALOS-2 PALSAR-2 ScanSAR...")
    try:
        palsar_collection_2023_2024 = ee.ImageCollection('JAXA/ALOS/PALSAR-2/Level2_2/ScanSAR') \
            .filterBounds(study_area_buffered) \
            .filterDate('2023-01-01', '2024-12-31')
        collection_size = palsar_collection_2023_2024.size().getInfo()
        data_source = "ALOS-2 PALSAR-2 ScanSAR"
        print(f"✓ ALOS-2 PALSAR-2 ScanSAR collection found")
    except Exception as e:
        print(f"✗ ALOS-2 PALSAR-2 ScanSAR: {str(e)[:100]}")
        collection_size = 0

# If no ALOS-2 data, try ALOS-4 PALSAR-3
if collection_size == 0:
    print(f"Trying ALOS-4 PALSAR-3...")
    try:
        palsar_collection_2023_2024 = ee.ImageCollection('JAXA/ALOS/PALSAR-3/YEARLY/SAR') \
            .filterBounds(study_area_buffered) \
            .filterDate('2023-01-01', '2024-12-31')
        collection_size = palsar_collection_2023_2024.size().getInfo()
        data_source = "ALOS-4 PALSAR-3"
        print(f"✓ ALOS-4 PALSAR-3 collection found")
    except:
        print("✗ ALOS-4 PALSAR-3: Collection not found in GEE")
        collection_size = 0

# If still no data, try original ALOS PALSAR
if collection_size == 0:
    print("Trying original ALOS PALSAR (legacy)...")
    try:
        palsar_collection_2023_2024 = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR') \
            .filterBounds(study_area_buffered) \
            .filterDate('2023-01-01', '2024-12-31')
        collection_size = palsar_collection_2023_2024.size().getInfo()
        data_source = "ALOS PALSAR (original/legacy)"
        print(f"✓ ALOS PALSAR (original) collection found")
    except Exception as e:
        print(f"✗ ALOS PALSAR (original): {str(e)[:100]}")
        collection_size = 0

print(f"\nCollection: {data_source}")
print(f"Images found for 2023-2024: {collection_size}")

# Show dates if data found
if collection_size > 0:
    dates = palsar_collection_2023_2024.aggregate_array('system:time_start').getInfo()
    sorted_dates = sorted(dates)
    print(f"\n📅 Available image dates:")
    for i, date in enumerate(sorted_dates, 1):
        date_str = ee.Date(date).format('YYYY-MM-dd').getInfo()
        print(f"   {i}. {date_str}")
    
    earliest = ee.Date(sorted_dates[0]).format('YYYY-MM-dd').getInfo()
    latest = ee.Date(sorted_dates[-1]).format('YYYY-MM-dd').getInfo()
    print(f"\n   Date range: {earliest} to {latest}")

# If no data, use latest available from any source
actual_period_label = "2023-2024"
if collection_size == 0:
    print(f"\n⚠️  No {data_source} data for 2023-2024, checking for latest available data...")
    
    # Try to get latest from ALOS-2 SAR_EPOCH first
    try:
        latest_alos2_epoch = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH') \
            .filterBounds(study_area_buffered) \
            .sort('system:time_start', False) \
            .limit(5)
        if latest_alos2_epoch.size().getInfo() > 0:
            palsar_collection_2023_2024 = latest_alos2_epoch
            data_source = "ALOS-2 PALSAR-2 (SAR_EPOCH - latest available)"
        else:
            raise Exception("No ALOS-2 EPOCH data")
    except:
        # Fall back to original PALSAR
        try:
            palsar_collection_2023_2024 = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR') \
                .filterBounds(study_area_buffered) \
                .sort('system:time_start', False) \
                .limit(5)
            data_source = "ALOS PALSAR (latest available)"
        except Exception as e:
            print(f"Error accessing PALSAR collections: {str(e)[:200]}")
            raise
    
    # Get actual dates being used
    actual_dates = palsar_collection_2023_2024.aggregate_array('system:time_start').getInfo()
    if len(actual_dates) > 0:
        sorted_actual_dates = sorted(actual_dates)
        earliest_actual = ee.Date(sorted_actual_dates[0]).format('YYYY-MM-dd').getInfo()
        latest_actual = ee.Date(sorted_actual_dates[-1]).format('YYYY-MM-dd').getInfo()
        actual_period_label = f"{earliest_actual} to {latest_actual} (Latest Available)"
        print(f"\n📅 Using fallback data from {data_source}:")
        for i, date in enumerate(sorted_actual_dates, 1):
            date_str = ee.Date(date).format('YYYY-MM-dd').getInfo()
            print(f"   {i}. {date_str}")
        print(f"\n   Date range: {earliest_actual} to {latest_actual}")

# Create median composite
palsar_2023_2024 = palsar_collection_2023_2024.median().clip(study_area_buffered)

# Convert to dB scale
# Note: SAR_EPOCH may have different band names or scaling
# Check available bands first
available_bands = palsar_2023_2024.bandNames().getInfo()
print(f"\nAvailable bands: {available_bands}")

# Handle different band naming conventions
if 'HH' in available_bands and 'HV' in available_bands:
    # Standard HH/HV bands
    palsar_2023_2024_db = palsar_2023_2024.select(['HH', 'HV']).multiply(ee.Image.constant(0.1)).log10().multiply(10)
elif 'hh' in available_bands and 'hv' in available_bands:
    # Lowercase band names
    palsar_2023_2024_db = palsar_2023_2024.select(['hh', 'hv']).multiply(ee.Image.constant(0.1)).log10().multiply(10).rename(['HH', 'HV'])
else:
    # Try to use first two bands if HH/HV not found
    print("⚠️  Warning: HH/HV bands not found, using first available bands")
    band_list = available_bands[:2]
    palsar_2023_2024_db = palsar_2023_2024.select(band_list).multiply(ee.Image.constant(0.1)).log10().multiply(10).rename(['HH', 'HV'])

# Add to map
Map.addLayer(palsar_2023_2024_db.select('HH'), {
    'min': -25,
    'max': 5,
    'palette': ['black', 'white']
}, f'PALSAR HH ({actual_period_label})', False)

Map.addLayer(palsar_2023_2024_db.select('HV'), {
    'min': -30,
    'max': 0,
    'palette': ['black', 'white']
}, f'PALSAR HV ({actual_period_label})', False)

# Create RGB composite
hh_hv_ratio_2023 = palsar_2023_2024_db.select('HH').divide(palsar_2023_2024_db.select('HV').add(0.01))
palsar_rgb_2023_2024 = ee.Image.cat([
    palsar_2023_2024_db.select('HH'),
    palsar_2023_2024_db.select('HV'),
    hh_hv_ratio_2023
]).rename(['R', 'G', 'B'])

Map.addLayer(palsar_rgb_2023_2024, {
    'min': -25,
    'max': 5
}, f'PALSAR RGB Composite ({actual_period_label})')

print(f"\n✓ PALSAR data loaded successfully!")
print(f"  Data source: {data_source}")
print(f"  Period label: {actual_period_label}")
if collection_size == 0:
    print(f"  ⚠️  Note: This is fallback data, not actual 2023-2024 data")
print("=" * 60)
Map


Collection size: 0
No PALSAR data for 2023-2024, using latest available data...
PALSAR data (2023-2024) loaded successfully!


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

In [66]:
# Lineament extraction using DEM-based methods
# Method 1: Edge detection on hillshade
def extract_lineaments_dem(elevation_img, slope_img, aspect_img, hillshade_img):
    """Extract lineaments from DEM using multiple methods"""
    
    # Method 1: Slope-based lineaments (steep slopes often indicate faults)
    slope_threshold = slope_img.gt(15)  # Threshold for steep slopes
    
    # Method 2: Aspect-based lineaments (linear features in aspect)
    # Convert aspect to radians and apply edge detection
    aspect_rad = aspect_img.multiply(ee.Image.constant(3.14159)).divide(180)
    
    # Method 3: Hillshade edge detection
    # Apply Sobel filter for edge detection
    # Create kernel for edge detection
    kernel = ee.Kernel.fixed(3, 3, [
        [-1, -1, -1],
        [0, 0, 0],
        [1, 1, 1]
    ])
    
    # Apply convolution for edge detection
    edges_x = hillshade_img.convolve(kernel)
    edges_y = hillshade_img.convolve(kernel.rotate(90))
    edges = edges_x.pow(2).add(edges_y.pow(2)).sqrt()
    
    # Threshold edges to identify lineaments
    lineament_edges = edges.gt(30).rename('Lineament_Edges')
    
    # Method 4: Curvature-based lineaments (gradient approach)
    # Smooth elevation first to reduce noise
    gaussian_filter = ee.Kernel.gaussian(radius=1.5, sigma=1, units='pixels')
    elev_smooth = elevation_img.convolve(gaussian_filter).resample('bilinear')
    
    # Calculate gradients and second derivatives
    grads = elev_smooth.gradient()
    second_grads = grads.gradient()
    d2z_dx2 = second_grads.select('x')
    d2z_dy2 = second_grads.select('y')
    curvature = d2z_dx2.abs().add(d2z_dy2.abs())
    
    # Threshold based on curvature magnitude (adjust threshold as needed)
    lineament_curvature = curvature.gt(0.02).rename('Lineament_Curvature')
    
    # Method 5: Aspect variance lineaments (high variance = linear features)
    # Use reduceNeighborhood with variance reducer (GEE doesn't have focal_variance)
    kernel_variance = ee.Kernel.square(radius=2, units='pixels')
    aspect_var = aspect_img.reduceNeighborhood(
        reducer=ee.Reducer.variance(),
        kernel=kernel_variance
    ).rename('aspect_variance')
    # Use a fixed threshold for aspect variance
    # For aspect (0-360 degrees), variance with 2-pixel radius typically ranges 0-50000
    # High variance (>10000) indicates linear features/lineaments
    # Adjust this threshold based on your study area characteristics
    aspect_var_threshold = ee.Image.constant(10000)  # Adjust as needed (typical range: 5000-20000)
    lineament_aspect_var = aspect_var.gt(aspect_var_threshold).rename('Lineament_Aspect_Var')
    
    # Combine all DEM-based methods
    lineament_combined = lineament_edges.add(lineament_curvature).add(lineament_aspect_var).gt(0).rename('Lineament_DEM')
    
    return {
        'lineament_edges': lineament_edges,
        'lineament_curvature': lineament_curvature,
        'lineament_aspect_var': lineament_aspect_var,
        'lineament_combined': lineament_combined,
        'edges': edges
    }

# Extract lineaments from DEM
dem_lineaments = extract_lineaments_dem(elevation, slope, aspect, hillshade)

# Visualize DEM-based lineaments
Map.addLayer(dem_lineaments['edges'], {
    'min': 0,
    'max': 100,
    'palette': ['black', 'white']
}, 'DEM Edge Detection', False)

Map.addLayer(dem_lineaments['lineament_edges'], {
    'min': 0,
    'max': 1,
    'palette': ['black', 'yellow']
}, 'Lineaments from Edges (DEM)')

Map.addLayer(dem_lineaments['lineament_curvature'], {
    'min': 0,
    'max': 1,
    'palette': ['black', 'cyan']
}, 'Lineaments from Curvature (DEM)', False)

Map.addLayer(dem_lineaments['lineament_combined'], {
    'min': 0,
    'max': 1,
    'palette': ['black', 'red']
}, 'Combined Lineaments (DEM)')

print("DEM-based lineaments extracted!")
Map


DEM-based lineaments extracted!


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

In [ ]:
# Lineament extraction from PALSAR radar data
# Radar backscatter can reveal structural features

def extract_lineaments_palsar(palsar_img):
    """Extract lineaments from PALSAR data"""
    
    # Use HH polarization for lineament detection (better for structural features)
    hh = palsar_img.select('HH')
    
    # Apply edge detection using Sobel filter
    kernel_x = ee.Kernel.fixed(3, 3, [
        [-1, 0, 1],
        [-2, 0, 2],
        [-1, 0, 1]
    ])
    
    kernel_y = ee.Kernel.fixed(3, 3, [
        [-1, -2, -1],
        [0, 0, 0],
        [1, 2, 1]
    ])
    
    # Apply convolution
    edges_x = hh.convolve(kernel_x)
    edges_y = hh.convolve(kernel_y)
    edges = edges_x.pow(2).add(edges_y.pow(2)).sqrt()
    
    # Method 2: HH/HV ratio (enhanced lineament detection)
    # Ratio helps identify structural features
    hh_hv_ratio = palsar_img.select('HH').divide(palsar_img.select('HV').add(0.01)).rename('HH_HV_Ratio')
    
    # Edge detection on ratio
    ratio_edges_x = hh_hv_ratio.convolve(kernel_x)
    ratio_edges_y = hh_hv_ratio.convolve(kernel_y)
    ratio_edges = ratio_edges_x.pow(2).add(ratio_edges_y.pow(2)).sqrt()
    
    # Combine HH edges and ratio edges for better detection
    combined_edges = edges.add(ratio_edges)
    
    # Threshold to identify lineaments using fixed threshold
    # PALSAR edge values typically range from 0-50 for combined edges
    # Use a threshold based on typical values (adjust as needed)
    # For 75th percentile equivalent, use ~30-40% of max range
    edge_threshold = ee.Image.constant(20)  # Adjust based on your data (typical range: 15-30)
    lineament_threshold = combined_edges.gt(edge_threshold).rename('Lineament_PALSAR')
    
    return {
        'edges': combined_edges,
        'lineament_threshold': lineament_threshold,
        'hh_hv_ratio': hh_hv_ratio
    }

# Extract lineaments from PALSAR radar data
# Deliverable: PALSAR-based lineament detection using radar backscatter analysis

print("=" * 60)
print("PALSAR LINEAMENT EXTRACTION")
print("=" * 60)
print("Method: Edge detection on HH polarization + HH/HV ratio analysis")
print("Output: Binary lineament map from radar data")
print("=" * 60)

# Check if 2023-2024 data exists
palsar_lineaments_2023_2024 = None

try:
    if 'palsar_2023_2024_db' in globals():
        bands_2023 = palsar_2023_2024_db.bandNames().getInfo()
        if len(bands_2023) > 0 and ('HH' in bands_2023 or 'hh' in bands_2023):
            palsar_lineaments_2023_2024 = extract_lineaments_palsar(palsar_2023_2024_db)
            print("✓ PALSAR lineaments extracted successfully")
        else:
            print("⚠️  PALSAR 2023-2024 data exists but has no valid bands")
    else:
        print("⚠️  PALSAR 2023-2024 data not available")
except Exception as e:
    print(f"⚠️  Error: {str(e)[:100]}")

# Visualize PALSAR-based lineaments
if palsar_lineaments_2023_2024 is not None:
    Map.addLayer(palsar_lineaments_2023_2024['edges'], {
        'min': 0,
        'max': 50,
        'palette': ['black', 'white']
    }, 'PALSAR Edge Detection (2023-2024)', False)

    Map.addLayer(palsar_lineaments_2023_2024['lineament_threshold'], {
        'min': 0,
        'max': 1,
        'palette': ['black', 'yellow']
    }, 'PALSAR Lineaments (2023-2024)')

    # Visualize HH/HV ratio for structural analysis
    Map.addLayer(palsar_lineaments_2023_2024['hh_hv_ratio'], {
        'min': 0,
        'max': 5,
        'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
    }, 'HH/HV Ratio - Structural Indicator (2023-2024)', False)
    
    print("\n✓ Deliverable: PALSAR lineament map created")
    print("  - Binary map: 1 = lineament detected, 0 = no lineament")
    print("  - Based on: Radar backscatter edges and polarization ratios")
else:
    print("⚠️  Cannot create PALSAR lineament map - no data available")

print("=" * 60)
Map


PALSAR-based lineaments extracted!


Map(bottom=127778.0, center=[4.92951505644009, 29.426879882812504], controls=(WidgetControl(options=['position…

In [ ]:
# Combine DEM and PALSAR lineaments for comprehensive lineament map
# Enhanced approach: DEM curvature + PALSAR edges/ratios + aspect variance
def combine_lineaments(dem_lineaments, palsar_lineaments, aspect_variance_img):
    """Combine DEM and PALSAR lineaments with enhanced confidence"""
    
    # Extract individual components from DEM (always available)
    dem_curvature = dem_lineaments['lineament_curvature'].unmask(0)
    dem_aspect_var = dem_lineaments['lineament_aspect_var'].unmask(0)
    dem_edges = dem_lineaments['lineament_edges'].unmask(0)
    
    # Normalize aspect variance to 0-1 range using unitScale
    # Aspect variance typically ranges from 0 to ~50000 for 2-pixel radius
    aspect_var_norm = aspect_variance_img.unitScale(0, 50000).clamp(0, 1)
    
    # Handle PALSAR data (may be None if not available)
    if palsar_lineaments is not None:
        palsar_edges = palsar_lineaments['edges'].unmask(0)
        palsar_threshold = palsar_lineaments['lineament_threshold'].unmask(0)
        
        # Normalize PALSAR edges to 0-1 range using unitScale (server-side operation)
        # PALSAR edges typically range from -50 to 50 for edge detection
        palsar_edges_norm = palsar_edges.unitScale(-50, 50).clamp(0, 1)
        
        # Combine with weights for higher confidence detection
        # DEM curvature: 0.3, PALSAR edges: 0.3, Aspect variance: 0.2, DEM edges: 0.2
        combined_score = (
            dem_curvature.multiply(0.3)
            .add(palsar_edges_norm.multiply(0.3))
            .add(aspect_var_norm.multiply(0.2))
            .add(dem_edges.multiply(0.2))
        )
    else:
        # If no PALSAR data, use only DEM and aspect variance
        # Adjust weights: DEM curvature: 0.4, Aspect variance: 0.3, DEM edges: 0.3
        combined_score = (
            dem_curvature.multiply(0.4)
            .add(aspect_var_norm.multiply(0.3))
            .add(dem_edges.multiply(0.3))
        )
    
    # Threshold for final lineament map (adjust threshold as needed)
    combined = combined_score.gt(0.4).rename('Combined_Lineaments')
    
    # Create confidence map (0-1 scale)
    confidence = combined_score.rename('Lineament_Confidence')
    
    return combined.addBands(confidence)

# Combine DEM and PALSAR lineaments for enhanced detection
# Deliverable: High-confidence lineament map with confidence scores

print("=" * 60)
print("COMBINED LINEAMENT MAP (SRTM DEM + PALSAR 2023-2024)")
print("=" * 60)
print("Method: Multi-source fusion with weighted combination")
print("  - DEM curvature (30%): Topographic lineaments")
print("  - PALSAR edges (30%): Radar-detected lineaments")
print("  - Aspect variance (20%): Linear topographic features")
print("  - DEM edges (20%): Hillshade-based edges")
print("=" * 60)

combined_lineaments_2023_2024 = None

# Create combined lineaments
if palsar_lineaments_2023_2024 is not None:
    combined_lineaments_2023_2024 = combine_lineaments(
        dem_lineaments,
        palsar_lineaments_2023_2024,
        aspect_variance
    )
    print("✓ Combined lineament map created (SRTM DEM + PALSAR)")
else:
    # Use DEM-only if PALSAR not available
    combined_lineaments_2023_2024 = combine_lineaments(
        dem_lineaments,
        None,
        aspect_variance
    )
    print("⚠️  Combined lineament map created (DEM-only, no PALSAR)")

# Visualize combined lineaments
if combined_lineaments_2023_2024 is not None:
    Map.addLayer(combined_lineaments_2023_2024.select('Combined_Lineaments'), {
        'min': 0,
        'max': 1,
        'palette': ['black', 'red']
    }, 'Combined Lineaments (SRTM + PALSAR)')

    Map.addLayer(combined_lineaments_2023_2024.select('Lineament_Confidence'), {
        'min': 0,
        'max': 1,
        'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
    }, 'Lineament Confidence Score', False)
    
    print("\n✓ Deliverable: Combined lineament map created")
    print("  - Binary map: 1 = lineament detected, 0 = no lineament")
    print("  - Confidence: 0-1 scale (higher = more reliable detection)")
    print("  - Based on: Agreement between DEM and PALSAR sources")
else:
    print("⚠️  Cannot create combined lineament map")

print("=" * 60)
Map


Combined lineament maps created!


Map(bottom=127778.0, center=[4.92951505644009, 29.426879882812504], controls=(WidgetControl(options=['position…

## Lineament Density Analysis


In [ ]:
# Calculate lineament density (lineaments per unit area)
def calculate_lineament_density(lineament_img, radius=500):
    """Calculate lineament density using focal statistics"""
    
    # Create a circular kernel for the moving window
    # Convert radius from meters to pixels (assuming 30m resolution)
    radius_pixels = radius / 30
    kernel = ee.Kernel.circle(radius=radius_pixels, units='pixels')
    
    # Count lineaments in a moving window using reduceNeighborhood
    lineament_count = lineament_img.select('Combined_Lineaments').reduceNeighborhood(
        reducer=ee.Reducer.sum(),
        kernel=kernel
    )
    
    # Calculate area in square kilometers (π * r²)
    area_km2 = 3.14159 * radius * radius / 1000000
    
    # Calculate density (lineaments per km²)
    lineament_density = lineament_count.divide(ee.Image.constant(area_km2)).rename('Lineament_Density')
    
    return lineament_density

# Calculate lineament density
# Deliverable: Lineament density map (lineaments per square kilometer)

print("=" * 60)
print("LINEAMENT DENSITY ANALYSIS")
print("=" * 60)
print("Definition: Number of lineaments per square kilometer")
print("Method: Moving window analysis (500m radius)")
print("Interpretation:")
print("  - High density (>5/km²): Zones of high structural complexity")
print("  - Medium density (2-5/km²): Moderate structural activity")
print("  - Low density (<2/km²): Stable areas with fewer fractures")
print("=" * 60)

lineament_density_2023_2024 = None

if combined_lineaments_2023_2024 is not None:
    lineament_density_2023_2024 = calculate_lineament_density(combined_lineaments_2023_2024)
    
    # Visualize lineament density
    Map.addLayer(lineament_density_2023_2024, {
        'min': 0,
        'max': 10,
        'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
    }, 'Lineament Density (lineaments/km²)')
    
    print("\n✓ Deliverable: Lineament density map created")
    print("  - Units: Lineaments per square kilometer")
    print("  - Range: 0-10+ lineaments/km²")
    print("  - Use: Identify zones of structural complexity for mineral exploration")
else:
    print("⚠️  Cannot calculate density - no lineament data available")

print("=" * 60)
Map


Lineament density calculated!


Map(bottom=127778.0, center=[4.92951505644009, 29.426879882812504], controls=(WidgetControl(options=['position…

## Lineament Orientation Analysis


In [ ]:
# Analyze lineament orientations (rose diagram data)
# Extract lineament directions from aspect and edge orientations

def analyze_lineament_orientation(lineament_img, aspect_img):
    """Analyze lineament orientations"""
    
    # Use aspect where lineaments exist
    lineament_aspect = aspect_img.updateMask(lineament_img.select('Combined_Lineaments').gt(0))
    
    # Classify into orientation classes (N, NE, E, SE, S, SW, W, NW)
    # Convert to 8 classes (45-degree intervals)
    orientation_classes = lineament_aspect.divide(45).int().mod(8).rename('Orientation_Class')
    
    # Create orientation map
    orientation_map = orientation_classes.updateMask(lineament_img.select('Combined_Lineaments').gt(0))
    
    return orientation_map

# Analyze lineament orientations
# Deliverable: Lineament orientation map showing dominant structural trends

print("=" * 60)
print("LINEAMENT ORIENTATION ANALYSIS")
print("=" * 60)
print("Definition: Directional classification of lineament trends")
print("Method: Aspect-based classification into 8 directional classes")
print("Orientation Classes:")
print("  0 = North (N)      - 0°-45°")
print("  1 = Northeast (NE) - 45°-90°")
print("  2 = East (E)       - 90°-135°")
print("  3 = Southeast (SE)- 135°-180°")
print("  4 = South (S)     - 180°-225°")
print("  5 = Southwest (SW)- 225°-270°")
print("  6 = West (W)       - 270°-315°")
print("  7 = Northwest (NW) - 315°-360°")
print("=" * 60)

lineament_orientation_2023_2024 = None

if combined_lineaments_2023_2024 is not None:
    lineament_orientation_2023_2024 = analyze_lineament_orientation(
        combined_lineaments_2023_2024,
        aspect
    )
    
    # Visualize orientations
    Map.addLayer(lineament_orientation_2023_2024, {
        'min': 0,
        'max': 7,
        'palette': ['red', 'orange', 'yellow', 'green', 'cyan', 'blue', 'purple', 'magenta']
    }, 'Lineament Orientation (8-directional)')
    
    print("\n✓ Deliverable: Lineament orientation map created")
    print("  - Shows: Dominant structural trends in the study area")
    print("  - Use: Identify preferred fracture orientations for mineral exploration")
    print("  - Interpretation: Rose diagrams can be generated from this data")
else:
    print("⚠️  Cannot analyze orientation - no lineament data available")

print("=" * 60)
Map


Lineament orientation analysis completed!
Note: 0=N, 1=NE, 2=E, 3=SE, 4=S, 5=SW, 6=W, 7=NW


Map(bottom=127778.0, center=[4.92951505644009, 29.426879882812504], controls=(WidgetControl(options=['position…

## Interpreting Lineament Maps for Structural Features

### How to Identify Different Structural Features

**1. Faults (Major Structural Discontinuities)**
- **Characteristics:**
  - Long, continuous linear features (>1 km)
  - High confidence scores (>0.7)
  - Often appear in parallel sets or conjugate pairs
  - May show topographic offsets in DEM
  - Strong radar backscatter contrast in PALSAR
- **Look for:** Straight or slightly curved lines with consistent orientation
- **Mineral Exploration:** Faults can act as conduits for mineralizing fluids

**2. Fractures/Joints (Smaller Discontinuities)**
- **Characteristics:**
  - Shorter lineaments (<1 km)
  - Often occur in sets with similar orientations
  - Medium confidence scores (0.4-0.7)
  - May form networks or grids
- **Look for:** Clustered short lineaments with similar directions
- **Mineral Exploration:** Fracture networks can host mineralization

**3. Zones of Structural Complexity**
- **Characteristics:**
  - High lineament density (>5/km²)
  - Multiple intersecting lineament orientations
  - High confidence areas with overlapping detections
  - Complex patterns (not simple linear features)
- **Look for:** Areas with dense, intersecting lineaments in density map
- **Mineral Exploration:** These zones often indicate favorable areas for mineralization

**4. Shear Zones**
- **Characteristics:**
  - Broad zones with multiple parallel lineaments
  - Consistent orientation over large areas
  - High aspect variance
  - May show topographic lineations
- **Look for:** Parallel lineament sets with consistent orientation
- **Mineral Exploration:** Shear zones are important for gold and base metal deposits

**5. Dykes/Intrusive Features**
- **Characteristics:**
  - Linear features with consistent orientation
  - May cross-cut other structures
  - Strong radar signature (high HH/HV ratio)
  - Topographic ridges or valleys
- **Look for:** Linear features that cut across regional structures
- **Mineral Exploration:** Can indicate pathways for mineralizing fluids

### Interpretation Guidelines

**Using the Confidence Map:**
- **High confidence (>0.7):** Reliable detection - likely real structural feature
- **Medium confidence (0.4-0.7):** Probable feature - verify with field work
- **Low confidence (<0.4):** Uncertain - may be noise or subtle features

**Using the Density Map:**
- **High density zones (>5/km²):** Focus areas for detailed exploration
- **Medium density (2-5/km²):** Moderate structural activity
- **Low density (<2/km²):** Stable areas, less favorable for mineralization

**Using the Orientation Map:**
- **Dominant orientations:** Indicate regional stress directions
- **Multiple orientations:** Suggest complex structural history
- **Preferred orientations:** Can guide drilling direction


In [ ]:
# Final Lineament Map Summary and Interpretation Guide
# Deliverable: Complete lineament analysis summary with interpretation tools

print("=" * 60)
print("LINEAMENT ANALYSIS SUMMARY")
print("=" * 60)
print("Data Sources: SRTM 30m DEM + PALSAR 2023-2024")
print("Analysis Period: 2023-2024")
print("=" * 60)

if combined_lineaments_2023_2024 is not None:
    # Show final combined lineament map
    Map.addLayer(combined_lineaments_2023_2024.select('Combined_Lineaments'), {
        'min': 0,
        'max': 1,
        'palette': ['black', 'red']
    }, 'Final Lineament Map (SRTM + PALSAR)')
    
    # Show confidence map
    Map.addLayer(combined_lineaments_2023_2024.select('Lineament_Confidence'), {
        'min': 0,
        'max': 1,
        'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
    }, 'Detection Confidence', False)
    
    print("\n✓ Final Deliverables Created:")
    print("  1. Combined Lineament Map: Binary detection map")
    print("  2. Confidence Map: Reliability score (0-1)")
    print("  3. Lineament Density: Lineaments per km²")
    print("  4. Lineament Orientation: 8-directional classification")
    print("  5. DEM-based Lineaments: Topographic features")
    print("  6. PALSAR-based Lineaments: Radar-detected features")
    
    print("\n" + "=" * 60)
    print("INTERPRETATION GUIDE")
    print("=" * 60)
    print("How to identify structural features on the map:")
    print("\n1. FAULTS (Major structures):")
    print("   - Look for: Long continuous red lines (>1 km)")
    print("   - Check confidence: High confidence (orange/red) = reliable")
    print("   - Check orientation: Consistent direction over large area")
    print("   - Use: Identify major structural controls for mineralization")
    
    print("\n2. FRACTURES (Smaller structures):")
    print("   - Look for: Shorter red lines (<1 km) in clusters")
    print("   - Check density: Medium-high density zones (2-5/km²)")
    print("   - Check orientation: Multiple similar orientations = fracture sets")
    print("   - Use: Identify fracture networks that can host mineralization")
    
    print("\n3. ZONES OF STRUCTURAL COMPLEXITY (High priority):")
    print("   - Look for: Areas with dense intersecting lineaments")
    print("   - Check density: High density (>5/km²) = red/orange on density map")
    print("   - Check confidence: High confidence with multiple detections")
    print("   - Use: Primary targets for mineral exploration")
    
    print("\n4. SHEAR ZONES:")
    print("   - Look for: Broad zones with parallel lineament sets")
    print("   - Check orientation: Consistent orientation over large area")
    print("   - Check density: Medium-high density with uniform pattern")
    print("   - Use: Important for gold and base metal deposits")
    
    print("\n5. INTERPRETATION WORKFLOW:")
    print("   Step 1: View 'Final Lineament Map' - identify all lineaments")
    print("   Step 2: Overlay 'Detection Confidence' - focus on high confidence areas")
    print("   Step 3: Check 'Lineament Density' - identify high density zones")
    print("   Step 4: Check 'Lineament Orientation' - understand structural trends")
    print("   Step 5: Cross-reference with DEM and PALSAR layers for validation")
    
    print("\n  All maps ready for export and further analysis")
else:
    print("⚠️  No lineament data available")

print("=" * 60)
Map


Temporal change analysis completed!


Map(bottom=127778.0, center=[4.92951505644009, 29.426879882812504], controls=(WidgetControl(options=['position…

In [ ]:
# Automatic Classification of Structural Features
# Classifies lineaments into: Faults, Fractures, Structural Complexity Zones, Shear Zones

print("=" * 60)
print("AUTOMATIC STRUCTURAL FEATURE CLASSIFICATION")
print("=" * 60)
print("Classifying lineaments into structural feature types...")
print("=" * 60)

def classify_structural_features(lineament_map, confidence_map, density_map, orientation_map):
    """
    Classify structural features based on characteristics:
    - Faults: High confidence (>0.7), continuous features
    - Fractures: Medium confidence (0.4-0.7), shorter features
    - Structural Complexity: High density (>5/km²) with multiple orientations
    - Shear Zones: Medium-high density (2-5/km²) with consistent orientation
    - Dykes: High confidence, cross-cutting features
    """
    
    lineaments = lineament_map.select('Combined_Lineaments')
    confidence = confidence_map.select('Lineament_Confidence')
    
    # 1. FAULTS: High confidence lineaments (>0.7)
    # Use morphological operations to identify continuous features
    # Apply dilation to connect nearby pixels (simulates continuous lineaments)
    fault_kernel = ee.Kernel.circle(radius=3, units='pixels')  # ~90m at 30m resolution
    dilated_lineaments = lineaments.focal_max(kernel=fault_kernel, iterations=2)
    faults = dilated_lineaments.And(confidence.gt(0.7)).rename('Faults')
    
    # 2. FRACTURES: Medium confidence (0.4-0.7), not classified as faults
    fractures = lineaments.And(confidence.gte(0.4)).And(confidence.lt(0.7)).And(faults.eq(0)).rename('Fractures')
    
    # 3. ZONES OF STRUCTURAL COMPLEXITY: High density (>5/km²) with multiple lineaments
    structural_complexity = density_map.gt(5).And(lineaments.gt(0)).rename('Structural_Complexity')
    
    # 4. SHEAR ZONES: Medium-high density (2-5/km²) with consistent orientation
    # Identify areas with uniform orientation (low orientation variance)
    # Calculate orientation variance using reduceNeighborhood
    orientation_kernel = ee.Kernel.circle(radius=5, units='pixels')  # ~150m radius
    orientation_variance = orientation_map.reduceNeighborhood(
        reducer=ee.Reducer.variance(),
        kernel=orientation_kernel
    )
    # Low variance = consistent orientation (shear zones)
    # High variance = multiple orientations (not shear zones)
    low_orientation_variance = orientation_variance.lt(2)  # Threshold for consistent orientation
    shear_zones = density_map.gte(2).And(density_map.lte(5)).And(low_orientation_variance).And(lineaments.gt(0)).rename('Shear_Zones')
    
    # 5. DYKES/INTRUSIVE FEATURES: High confidence, cross-cutting patterns
    # Identify lineaments that are isolated or cross-cut other structures
    # Use focal statistics to find isolated high-confidence features
    isolated_kernel = ee.Kernel.circle(radius=2, units='pixels')
    neighbor_count = lineaments.reduceNeighborhood(
        reducer=ee.Reducer.count(),
        kernel=isolated_kernel
    )
    # Isolated or sparse lineaments with high confidence = potential dykes
    isolated_high_conf = neighbor_count.lte(3).And(confidence.gt(0.6)).And(lineaments.gt(0)).rename('Dykes')
    
    # Combine all classifications
    classified = ee.Image.cat([
        faults,
        fractures,
        structural_complexity,
        shear_zones,
        isolated_high_conf
    ])
    
    return classified

if (combined_lineaments_2023_2024 is not None and 
    lineament_density_2023_2024 is not None and 
    lineament_orientation_2023_2024 is not None):
    
    # Classify structural features
    structural_features = classify_structural_features(
        combined_lineaments_2023_2024,
        combined_lineaments_2023_2024,
        lineament_density_2023_2024,
        lineament_orientation_2023_2024
    )
    
    # Visualize each feature type
    Map.addLayer(structural_features.select('Faults'), {
        'min': 0,
        'max': 1,
        'palette': ['transparent', 'red']
    }, '1. Faults (High Confidence, Continuous)', False)
    
    Map.addLayer(structural_features.select('Fractures'), {
        'min': 0,
        'max': 1,
        'palette': ['transparent', 'yellow']
    }, '2. Fractures (Medium Confidence)', False)
    
    Map.addLayer(structural_features.select('Structural_Complexity'), {
        'min': 0,
        'max': 1,
        'palette': ['transparent', 'orange']
    }, '3. Structural Complexity Zones (>5/km²)', False)
    
    Map.addLayer(structural_features.select('Shear_Zones'), {
        'min': 0,
        'max': 1,
        'palette': ['transparent', 'purple']
    }, '4. Shear Zones (2-5/km², Consistent Orientation)', False)
    
    Map.addLayer(structural_features.select('Dykes'), {
        'min': 0,
        'max': 1,
        'palette': ['transparent', 'cyan']
    }, '5. Dykes/Intrusive Features (Isolated, High Confidence)', False)
    
    # Create composite classification map
    # Assign different values to each class for visualization
    fault_class = structural_features.select('Faults').multiply(1)
    fracture_class = structural_features.select('Fractures').multiply(2)
    complexity_class = structural_features.select('Structural_Complexity').multiply(3)
    shear_class = structural_features.select('Shear_Zones').multiply(4)
    dyke_class = structural_features.select('Dykes').multiply(5)
    
    # Combine all classes (priority: complexity > faults > shear > fractures > dykes)
    composite_classification = (
        complexity_class
        .add(fault_class.multiply(complexity_class.eq(0)))
        .add(shear_class.multiply(complexity_class.eq(0).And(fault_class.eq(0))))
        .add(fracture_class.multiply(complexity_class.eq(0).And(fault_class.eq(0)).And(shear_class.eq(0))))
        .add(dyke_class.multiply(complexity_class.eq(0).And(fault_class.eq(0)).And(shear_class.eq(0)).And(fracture_class.eq(0))))
    ).rename('Structural_Classification')
    
    Map.addLayer(composite_classification, {
        'min': 0,
        'max': 5,
        'palette': ['transparent', 'red', 'yellow', 'orange', 'purple', 'cyan']
    }, 'Structural Feature Classification (Composite)')
    
    print("\n✓ Structural features classified and mapped:")
    print("  1. Faults (Red): High confidence, continuous lineaments")
    print("  2. Fractures (Yellow): Medium confidence, shorter features")
    print("  3. Structural Complexity (Orange): High density zones (>5/km²)")
    print("  4. Shear Zones (Purple): Medium density, consistent orientation")
    print("  5. Dykes/Intrusive (Cyan): Isolated, high confidence features")
    print("\n  Composite map shows all classifications together")
    print("  Priority order: Complexity > Faults > Shear > Fractures > Dykes")
    
    # Store for export
    structural_features_classified = structural_features
    
else:
    print("⚠️  Cannot classify structural features - missing required data")
    structural_features_classified = None

print("=" * 60)
Map


## Integrated Analysis: Structural Features + Mineralization

This section combines structural features (from PALSAR/SRTM analysis) with alteration/mineralization zones (from ASTER analysis) to identify priority exploration targets.

**Key Integration Concepts:**
- **Structural Control:** Faults and fractures often control mineralization
- **Alteration Zones:** Clay, iron, and carbonate alteration indicate mineralizing processes
- **Priority Targets:** High structural complexity + high alteration = best exploration targets
- **Structural-Mineralization Relationships:** Understanding how structures relate to mineralization

**Note:** This analysis requires running both `mining-aster.ipynb` and `mining-palsar.ipynb` notebooks. The ASTER outputs (mineralization maps, alteration indices) should be available in the same kernel session.


In [ ]:
# Integrated Analysis: Structural Features + Mineralization
# Combines PALSAR/SRTM structural analysis with ASTER alteration/mineralization

print("=" * 60)
print("INTEGRATED STRUCTURAL-MINERALIZATION ANALYSIS")
print("=" * 60)
print("Combining structural features with alteration/mineralization zones")
print("=" * 60)

def create_integrated_targets(structural_features, lineament_density, mineralization_prob, alteration_zones=None):
    """
    Create integrated exploration target map combining:
    - Structural features (faults, fractures, complexity zones)
    - Lineament density
    - Mineralization probability
    - Alteration zones (if available)
    """
    
    # 1. PRIORITY TARGETS: High structural complexity + High mineralization
    # High density (>5/km²) + High mineralization probability (>0.6)
    high_density = lineament_density.gt(5)
    high_mineralization = mineralization_prob.gt(0.6)
    priority_targets = high_density.And(high_mineralization).rename('Priority_Targets')
    
    # 2. STRUCTURAL CONTROL ZONES: Faults intersecting mineralization
    # Faults + Medium-high mineralization (>0.4)
    faults = structural_features.select('Faults')
    medium_mineralization = mineralization_prob.gt(0.4)
    structural_control = faults.And(medium_mineralization).rename('Structural_Control')
    
    # 3. ALTERATION-STRUCTURE INTERSECTIONS: Alteration along lineaments
    # High lineament density + High mineralization = favorable zones
    favorable_zones = lineament_density.gt(3).And(mineralization_prob.gt(0.5)).rename('Favorable_Zones')
    
    # 4. EXPLORATION TARGET SCORE: Weighted combination
    # Normalize inputs to 0-1 scale
    density_norm = lineament_density.unitScale(0, 10)  # Normalize 0-10/km² to 0-1
    mineral_norm = mineralization_prob  # Already 0-1
    
    # Structural complexity score (from structural features)
    complexity_score = structural_features.select('Structural_Complexity').add(
        structural_features.select('Faults').multiply(0.8)
    ).add(
        structural_features.select('Shear_Zones').multiply(0.5)
    ).unitScale(0, 2.3)  # Max possible value ~2.3
    
    # Combined exploration score (weighted)
    exploration_score = (
        density_norm.multiply(0.3)  # 30% weight on structure density
        .add(mineral_norm.multiply(0.4))  # 40% weight on mineralization
        .add(complexity_score.multiply(0.3))  # 30% weight on structural complexity
    ).rename('Exploration_Score')
    
    # Classify into target categories
    high_targets = exploration_score.gt(0.7).rename('High_Priority')
    medium_targets = exploration_score.gt(0.5).And(exploration_score.lte(0.7)).rename('Medium_Priority')
    low_targets = exploration_score.gt(0.3).And(exploration_score.lte(0.5)).rename('Low_Priority')
    
    # Combine all outputs
    integrated = ee.Image.cat([
        priority_targets,
        structural_control,
        favorable_zones,
        exploration_score,
        high_targets,
        medium_targets,
        low_targets
    ])
    
    return integrated

# Check if ASTER outputs are available
try:
    # Try to access ASTER mineralization outputs
    # These should be created in mining-aster.ipynb
    if 'mineralisation_2023_2024' in globals():
        mineralization_map = mineralisation_2023_2024.select('Mineralisation_Probability')
        print("✓ ASTER mineralization data found")
        
        # Check for structural features
        if ('structural_features_classified' in globals() and 
            structural_features_classified is not None and
            lineament_density_2023_2024 is not None):
            
            # Create integrated analysis
            integrated_targets = create_integrated_targets(
                structural_features_classified,
                lineament_density_2023_2024,
                mineralization_map
            )
            
            # Visualize integrated targets
            Map.addLayer(integrated_targets.select('Exploration_Score'), {
                'min': 0,
                'max': 1,
                'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
            }, 'Exploration Target Score (0-1)', False)
            
            Map.addLayer(integrated_targets.select('High_Priority'), {
                'min': 0,
                'max': 1,
                'palette': ['transparent', 'red']
            }, 'High Priority Targets (>0.7)', False)
            
            Map.addLayer(integrated_targets.select('Medium_Priority'), {
                'min': 0,
                'max': 1,
                'palette': ['transparent', 'orange']
            }, 'Medium Priority Targets (0.5-0.7)', False)
            
            Map.addLayer(integrated_targets.select('Low_Priority'), {
                'min': 0,
                'max': 1,
                'palette': ['transparent', 'yellow']
            }, 'Low Priority Targets (0.3-0.5)', False)
            
            Map.addLayer(integrated_targets.select('Priority_Targets'), {
                'min': 0,
                'max': 1,
                'palette': ['transparent', 'magenta']
            }, 'Priority: High Density + High Mineralization', False)
            
            Map.addLayer(integrated_targets.select('Structural_Control'), {
                'min': 0,
                'max': 1,
                'palette': ['transparent', 'cyan']
            }, 'Structural Control: Faults + Mineralization', False)
            
            Map.addLayer(integrated_targets.select('Favorable_Zones'), {
                'min': 0,
                'max': 1,
                'palette': ['transparent', 'green']
            }, 'Favorable Zones: Density + Mineralization', False)
            
            print("\n✓ Integrated analysis completed!")
            print("\nDeliverables Created:")
            print("  1. Exploration Target Score: Weighted combination (0-1)")
            print("  2. High Priority Targets: Score >0.7 (red)")
            print("  3. Medium Priority Targets: Score 0.5-0.7 (orange)")
            print("  4. Low Priority Targets: Score 0.3-0.5 (yellow)")
            print("  5. Priority Targets: High density + High mineralization (magenta)")
            print("  6. Structural Control: Faults intersecting mineralization (cyan)")
            print("  7. Favorable Zones: Medium-high density + mineralization (green)")
            
            print("\n" + "=" * 60)
            print("INTERPRETATION GUIDE")
            print("=" * 60)
            print("Priority Target Identification:")
            print("  - High Priority (Red): Best exploration targets")
            print("    * High structural complexity (>5/km²)")
            print("    * High mineralization probability (>0.6)")
            print("    * Strong structural control")
            print("  - Medium Priority (Orange): Good exploration targets")
            print("    * Moderate structural complexity (3-5/km²)")
            print("    * Moderate mineralization (0.4-0.6)")
            print("  - Low Priority (Yellow): Potential targets")
            print("    * Lower scores but still favorable")
            print("\n  Structural Control Zones (Cyan):")
            print("    * Faults directly intersecting mineralization zones")
            print("    * Critical for understanding ore controls")
            print("\n  Priority Targets (Magenta):")
            print("    * Highest density zones with highest mineralization")
            print("    * Top priority for field investigation")
            
            # Store for export
            integrated_targets_2023_2024 = integrated_targets
            
        else:
            print("⚠️  Structural features not available. Run Cell 26 first.")
            integrated_targets_2023_2024 = None
            
    else:
        print("⚠️  ASTER mineralization data not found in current session.")
        print("   Please run mining-aster.ipynb first to generate:")
        print("   - mineralisation_2023_2024 (mineralization probability map)")
        print("   - Or manually load ASTER outputs if exported")
        print("\n   To use this analysis:")
        print("   1. Run mining-aster.ipynb (all cells)")
        print("   2. Keep the kernel running")
        print("   3. Run this cell in mining-palsar.ipynb")
        integrated_targets_2023_2024 = None
        
except NameError as e:
    print(f"⚠️  Error: {e}")
    print("   ASTER outputs not available. Please run mining-aster.ipynb first.")
    integrated_targets_2023_2024 = None

print("=" * 60)
Map


In [ ]:
# Export lineament maps
# Note: Uncomment and run to export to Google Drive

def export_lineament_maps():
    """Export all lineament analysis deliverables"""
    
    print("=" * 60)
    print("EXPORTING LINEAMENT ANALYSIS DELIVERABLES")
    print("=" * 60)
    print("Exporting to Google Drive...")
    print("=" * 60)
    
    # 1. DEM-based lineaments
    if 'dem_lineaments' in globals():
        geemap.ee_export_image(
            dem_lineaments['lineament_combined'],
            filename='01_lineaments_dem.tif',
            scale=30,
            region=study_area_buffered,
            file_per_band=False
        )
        print("✓ 01_lineaments_dem.tif - DEM-based lineaments")
    
    # 2. PALSAR-based lineaments
    if palsar_lineaments_2023_2024 is not None:
        geemap.ee_export_image(
            palsar_lineaments_2023_2024['lineament_threshold'],
            filename='02_lineaments_palsar.tif',
            scale=30,
            region=study_area_buffered,
            file_per_band=False
        )
        print("✓ 02_lineaments_palsar.tif - PALSAR-based lineaments")
    else:
        print("⚠️  Skipped: PALSAR lineaments (no data)")
    
    # 3. Combined lineaments (main deliverable)
    if combined_lineaments_2023_2024 is not None:
        geemap.ee_export_image(
            combined_lineaments_2023_2024,
            filename='03_lineaments_combined_srtm_palsar.tif',
            scale=30,
            region=study_area_buffered,
            file_per_band=False
        )
        print("✓ 03_lineaments_combined_srtm_palsar.tif - Combined lineament map + confidence")
    else:
        print("⚠️  Skipped: Combined lineaments (no data)")
    
    # 4. Lineament density
    if lineament_density_2023_2024 is not None:
        geemap.ee_export_image(
            lineament_density_2023_2024,
            filename='04_lineament_density_per_km2.tif',
            scale=30,
            region=study_area_buffered,
            file_per_band=False
        )
        print("✓ 04_lineament_density_per_km2.tif - Density map (lineaments/km²)")
    else:
        print("⚠️  Skipped: Lineament density (no data)")
    
    # 5. Lineament orientation
    if lineament_orientation_2023_2024 is not None:
        geemap.ee_export_image(
            lineament_orientation_2023_2024,
            filename='05_lineament_orientation_8dir.tif',
            scale=30,
            region=study_area_buffered,
            file_per_band=False
        )
        print("✓ 05_lineament_orientation_8dir.tif - Orientation map (8 directions)")
    else:
        print("⚠️  Skipped: Lineament orientation (no data)")
    
    # 6. Structural feature classification (if available)
    if 'structural_features_classified' in globals() and structural_features_classified is not None:
        geemap.ee_export_image(
            structural_features_classified,
            filename='06_structural_features_classified.tif',
            scale=30,
            region=study_area_buffered,
            file_per_band=False
        )
        print("✓ 06_structural_features_classified.tif - Classified structural features")
        print("  Bands: Faults, Fractures, Structural_Complexity, Shear_Zones, Dykes")
    else:
        print("⚠️  Skipped: Structural feature classification (run Cell 26 first)")
    
    # 7. Integrated targets (if available - requires ASTER outputs)
    if 'integrated_targets_2023_2024' in globals() and integrated_targets_2023_2024 is not None:
        geemap.ee_export_image(
            integrated_targets_2023_2024,
            filename='07_integrated_exploration_targets.tif',
            scale=30,
            region=study_area_buffered,
            file_per_band=False
        )
        print("✓ 07_integrated_exploration_targets.tif - Integrated structural + mineralization")
        print("  Bands: Priority_Targets, Structural_Control, Favorable_Zones,")
        print("         Exploration_Score, High_Priority, Medium_Priority, Low_Priority")
    else:
        print("⚠️  Skipped: Integrated targets (run Cell 28 first, requires ASTER outputs)")
    
    print("\n" + "=" * 60)
    print("EXPORT SUMMARY")
    print("=" * 60)
    print("All deliverables exported to Google Drive")
    print("Files ready for GIS analysis and interpretation")
    print("=" * 60)

print("Export functions defined. Uncomment and call to export maps.")
print("Example: export_lineament_maps()")


Export functions defined. Uncomment and call to export maps.
Example: export_lineament_maps()
